# Task 2- Argumentative Components Identification
* Binary Classification of Argumentative Sentences into claim and premises
* Following Haddadan et al. (2019)

## Requirements

### Installs

In [ ]:
!pip install torch transformers datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### General Requirements

In [ ]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login
import os
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import numpy as np
from torch.utils.data import DataLoader

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

## UNSC Dataset with Claim-Premise Annotations from Maria Poiaganova

In [ ]:
# Dataset retrieved from https://github.com/mpoiaganova/political-argument-mining
unsc_testing = pd.read_csv('sentence_full.csv', sep =',')
unsc_testing

,Sentence,Label,Components
0,"Once again, since the last briefing to the Cou...",claim,{'the situation regarding Ukraine has seriousl...
1,This is now the tenth time that the Council ha...,none,{}
2,The General Assembly also took up the matter o...,none,{}
3,"Following close to two weeks of relative calm,...",premise,{'Following close to two weeks of relative cal...
4,The individuals involved called for secession ...,premise,{'The individuals involved called for secessio...
...,...,...,...
4746,We welcome Italy's decision to designate dialo...,claim,"{""We welcome Italy's decision to designate dia..."
4747,China supports practical and effective coopera...,claim,{'China supports practical and effective coope...
4748,We welcome all the positive efforts being made...,claim,{'We welcome all the positive efforts being ma...
4749,We hope that all the parties concerned will wo...,claim,{'We hope that all the parties concerned will ...


In [ ]:

valid = ['claim', 'premise']
utest = unsc_testing.loc[(unsc_testing['Label'].isin(valid))]

In [ ]:
utest['Label'] = utest['Label'].str.replace('claim','Claim')
utest['Label'] = utest['Label'].str.replace('premise','Premise')

<ipython-input-57-68bc61df8b65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utest['Label'] = utest['Label'].str.replace('claim','Claim')
<ipython-input-57-68bc61df8b65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utest['Label'] = utest['Label'].str.replace('premise','Premise')


In [ ]:
utest

,Sentence,Label,Components
0,"Once again, since the last briefing to the Cou...",Claim,{'the situation regarding Ukraine has seriousl...
3,"Following close to two weeks of relative calm,...",Premise,{'Following close to two weeks of relative cal...
4,The individuals involved called for secession ...,Premise,{'The individuals involved called for secessio...
5,"In Donetsk, some of these individuals declared...",Premise,"{""In Donetsk, some of these individuals declar..."
6,It has now been reported that there have been ...,Premise,{'It has now been reported that there have bee...
...,...,...,...
4746,We welcome Italy's decision to designate dialo...,Claim,"{""We welcome Italy's decision to designate dia..."
4747,China supports practical and effective coopera...,Claim,{'China supports practical and effective coope...
4748,We welcome all the positive efforts being made...,Claim,{'We welcome all the positive efforts being ma...
4749,We hope that all the parties concerned will wo...,Claim,{'We hope that all the parties concerned will ...


In [ ]:
# Mapping labels to numeric values
label_mapping = {"Premise": 1, "Claim": 0}

utest['Component'] = utest['Label'].map(label_mapping)
utest

<ipython-input-59-059dfd080157>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utest['Component'] = utest['Label'].map(label_mapping)


,Sentence,Label,Components,Component
0,"Once again, since the last briefing to the Cou...",Claim,{'the situation regarding Ukraine has seriousl...,0
3,"Following close to two weeks of relative calm,...",Premise,{'Following close to two weeks of relative cal...,1
4,The individuals involved called for secession ...,Premise,{'The individuals involved called for secessio...,1
5,"In Donetsk, some of these individuals declared...",Premise,"{""In Donetsk, some of these individuals declar...",1
6,It has now been reported that there have been ...,Premise,{'It has now been reported that there have bee...,1
...,...,...,...,...
4746,We welcome Italy's decision to designate dialo...,Claim,"{""We welcome Italy's decision to designate dia...",0
4747,China supports practical and effective coopera...,Claim,{'China supports practical and effective coope...,0
4748,We welcome all the positive efforts being made...,Claim,{'We welcome all the positive efforts being ma...,0
4749,We hope that all the parties concerned will wo...,Claim,{'We hope that all the parties concerned will ...,0


In [ ]:
print(utest.shape)

(3814, 4)


In [ ]:
print(utest['Label'].value_counts())

Label
Claim      2077
Premise    1737
Name: count, dtype: int64


In [ ]:
# Rename 'Component' to 'label' for Hugging Face Trainer to recognize labels
utest_df = utest.rename(columns={"Component": "label"})
utest_df = utest.rename(columns={"Sentence": "Speech"})
utest_set = Dataset.from_pandas(utest_df)
utest_set_set = utest_set.map(tokenize_function, batched=True)

Map:   0%|          | 0/3814 [00:00<?, ? examples/s]

## Loading USElecDeb Dataset
* From Haddadan et al. (2019)

In [ ]:
# Dataset retrieved from https://github.com/mpoiaganova/political-argument-mining
usElecDeb = pd.read_csv('sentence_db_candidate.csv', sep =',')
usElecDeb

,Text,Part,Document,Order,Sentence,Start,End,Annotator,Tag,Component,Speech,Speaker,SpeakerType,Set,Date,Year,Name,MainTag
0,"CHENEY: Gwen, I want to thank you, and I want ...",1,30_2004,0,0,2101,2221,NaN,"{""O"": 27}",O,"Gwen, I want to thank you, and I want to than...",CHENEY,Candidate,TRAIN,05 Oct 2004,2004,Richard(Dick) B. Cheney,O
1,"It's a very important event, and they've done ...",1,30_2004,1,1,2221,2304,NaN,"{""O"": 19}",O,"It's a very important event, and they've done ...",CHENEY,Candidate,TRAIN,05 Oct 2004,2004,Richard(Dick) B. Cheney,O
2,It's important to look at all of our developme...,1,30_2004,2,2,2304,2418,NaN,"{""O"": 23}",O,It's important to look at all of our developme...,CHENEY,Candidate,TRAIN,05 Oct 2004,2004,Richard(Dick) B. Cheney,O
3,"And, after 9/11, it became clear that we had t...",1,30_2004,3,3,2418,2744,NaN,"{""O"": 16, ""Claim"": 50}",Claim,"And, after 9/11, it became clear that we had t...",CHENEY,Candidate,TRAIN,05 Oct 2004,2004,Richard(Dick) B. Cheney,Claim
4,And we also then finally had to stand up democ...,1,30_2004,4,4,2744,2974,NaN,"{""O"": 4, ""Claim"": 13, ""Premise"": 25}",Premise,And we also then finally had to stand up democ...,CHENEY,Candidate,TRAIN,05 Oct 2004,2004,Richard(Dick) B. Cheney,Mixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29616,And we'll continue to promote freedom around t...,162,31_2004,22,22,91397,91453,NaN,"{""O"": 2, ""Claim"": 9}",Claim,And we'll continue to promote freedom around t...,BUSH,Candidate,VALIDATION,08 Oct 2004,2004,George W. Bush,Claim
29617,Freedom is on the march.,162,31_2004,23,23,91454,91479,NaN,"{""Claim"": 5, ""O"": 1}",Claim,Freedom is on the march.,BUSH,Candidate,VALIDATION,08 Oct 2004,2004,George W. Bush,Claim
29618,"Tomorrow, Afghanistan will be voting for a pre...",162,31_2004,24,24,91479,91533,NaN,"{""Premise"": 9, ""O"": 1}",Premise,"Tomorrow, Afghanistan will be voting for a pre...",BUSH,Candidate,VALIDATION,08 Oct 2004,2004,George W. Bush,Premise
29619,"In Iraq, we'll be having free elections, and a...",162,31_2004,25,25,91533,91629,NaN,"{""Premise"": 9, ""O"": 4, ""Claim"": 8}",Premise,"In Iraq, we'll be having free elections, and a...",BUSH,Candidate,VALIDATION,08 Oct 2004,2004,George W. Bush,Mixed


In [ ]:
valid = ['Claim', 'Premise']
usElecDeb = usElecDeb.loc[(usElecDeb['Component'].isin(valid))]

In [ ]:
# Splitting USElecDeb dataset as Haddadan et al.(2019)
df_train = usElecDeb[usElecDeb['Set'] == 'TRAIN']
df_val = usElecDeb[usElecDeb['Set'] == 'VALIDATION']
df_test = usElecDeb[usElecDeb['Set'] == 'TEST']

df_train = df_train[['Speech', 'Component']]
df_val = df_val[['Speech', 'Component']]
df_test = df_test[['Speech', 'Component']]


In [ ]:
print(df_train.shape, df_val.shape, df_test.shape)

(10464, 2) (5241, 2) (6575, 2)


In [ ]:
# Printing label counts in test set
df_test['Component'].value_counts()

,count
Component,
Claim,3361
Premise,3214


In [ ]:
df_test['Component'].unique()

array(['Premise', 'Claim'], dtype=object)

In [ ]:
# Mapping label values to numeric value s
label_mapping = {"Premise": 1, "Claim": 0}


df_train['Component'] = df_train['Component'].map(label_mapping)
df_val['Component'] = df_val['Component'].map(label_mapping)
df_test['Component'] = df_test['Component'].map(label_mapping)

In [ ]:
df_test['Component'].value_counts()

,count
Component,
0,3361
1,3214


In [ ]:
# Renaming 'Component' to 'label' for Hugging Face Trainer to recognize labels
df_train = df_train.rename(columns={"Component": "label"})
df_val = df_val.rename(columns={"Component": "label"})
df_test = df_test.rename(columns={"Component": "label"})

In [ ]:
# Converting datasets into Hugging Face Dataset format
train_set = Dataset.from_pandas(df_train)
val_set = Dataset.from_pandas(df_val)
test_set = Dataset.from_pandas(df_test)


In [ ]:
test_set

Dataset({
    features: ['Speech', 'label', '__index_level_0__'],
    num_rows: 6575
})

## Loading RoBERTa Model

In [ ]:
# Setting up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Loading the Tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Defining function for tokenization
def tokenize_function(examples):
    return tokenizer(examples['Speech'], padding="max_length", truncation=True)

In [ ]:
# Tokenizing all sets (train, validation, and test)
tokenized_train_set = train_set.map(tokenize_function, batched=True)
tokenized_val_set = val_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

Map:   0%|          | 0/10464 [00:00<?, ? examples/s]

Map:   0%|          | 0/5241 [00:00<?, ? examples/s]

Map:   0%|          | 0/6575 [00:00<?, ? examples/s]

In [ ]:
# Loading the model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Defining the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate at the end of each epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=4,  # batch size for training
    per_device_eval_batch_size=4,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,                # Logging steps
    report_to="none",                # Disable wandb
)

# Customizing the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Defining compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
# Creating  Trainer
trainer = Trainer(
    model=model,                         # the pre-trained model
    args=training_args,                  # training arguments
    train_dataset=tokenized_train_set,   # tokenized training dataset
    eval_dataset=tokenized_val_set,      # tokenized validation dataset
    tokenizer=tokenizer,                 # tokenizer
    optimizers=(optimizer, None),        # passing the AdamW optimizer and using the default scheduler
    compute_metrics=compute_metrics      # metrics
)

<ipython-input-28-15fc8b33c7ef>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Fine-tuning the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.557500,0.674981,0.708643,0.703485,0.711960,0.708643
2,0.852900,0.906707,0.709216,0.702933,0.714351,0.709216
3,0.700600,1.160856,0.712841,0.710831,0.712642,0.712841


TrainOutput(global_step=7848, training_loss=0.5784889834037736, metrics={'train_runtime': 3976.6847, 'train_samples_per_second': 7.894, 'train_steps_per_second': 1.974, 'total_flos': 8259582249861120.0, 'train_loss': 0.5784889834037736, 'epoch': 3.0})

In [ ]:
# Saving fine-tuned model-
model.save_pretrained("ADD PATH") # ADD PATH
tokenizer.save_pretrained("ADD PATH") # ADD PATH

('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Experiments/Task 1 (arg vs non arg) and Task 2 (premise and claim )/Task 2 - Binary Classification (premise and claim)/fine_tuned_roberta_claim_premise_FV/tokenizer_config.json',
 '/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Experiments/Task 1 (arg vs non arg) and Task 2 (premise and claim )/Task 2 - Binary Classification (premise and claim)/fine_tuned_roberta_claim_premise_FV/special_tokens_map.json',
 '/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Experiments/Task 1 (arg vs non arg) and Task 2 (premise and claim )/Task 2 - Binary Classification (premise and claim)/fine_tuned_roberta_claim_premise_FV/vocab.json',
 '/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Experiments/Task 1 (arg vs non arg) and Task 2 (premise and claim )/Task 2 - Binary Classification (premise and claim)/fine_tuned_roberta_claim_premise_FV/merges.txt',
 '/content/drive/MyDrive/Master Thesis - 2024 2025/P

## Testing with USElecDeb

In [ ]:
# Creating test data loader
test_loader = DataLoader(tokenized_test_set, batch_size=16)

In [ ]:
# Formating tokenized test set
tokenized_test_set.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
# Function for predictions
def predict(model, dataloader):
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            predictions.append(logits.argmax(dim=-1).cpu().numpy())

    return predictions


In [ ]:
# Getting predictions
predictions = predict(model, test_loader)

In [ ]:
# COnverting predictions to flat list
predictions = [item for sublist in predictions for item in sublist]

In [ ]:
# Adding prections to test set Dataframe
df_test['Predicted_Label'] = predictions

In [ ]:
# Printing predictions
df_test

,Speech,label,Predicted_Label
1418,"In 20 days, you will help decide who will lead...",1,0
1420,"And you must ask yourself, do you know enough ...",1,1
1421,You should know as much as possible about each...,0,0
1422,Sometimes the views have been distorted.,1,1
1423,There's been millions and millions of dollars ...,0,1
...,...,...,...
27406,So I ask you to look at that.,1,1
27407,And you have to vote for somebody with a plan.,0,0
27408,That's what you have elections for.,1,1
27409,"If people would say, well, he got elected to d...",0,1


### Evaluation of test set predictions from USElectDeb

In [ ]:
# Getting true labels and predicted labels form test dataframe
y_true = df_test['label']
y_pred = df_test['Predicted_Label']

In [ ]:
# Calculating metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, pos_label=1)  # assuming 'Premise'=1 is the positive class
recall = recall_score(y_true, y_pred, pos_label=1)
f1 = f1_score(y_true, y_pred, pos_label=1)
conf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
# Printing the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Accuracy: 0.6952091254752851
Precision: 0.7232472324723247
Recall: 0.6098319850653391
F1 Score: 0.6617150573936529
Confusion Matrix:
 [[2611  750]
 [1254 1960]]

Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.78      0.72      3361
           1       0.72      0.61      0.66      3214

    accuracy                           0.70      6575
   macro avg       0.70      0.69      0.69      6575
weighted avg       0.70      0.70      0.69      6575



## Testing on UNSC dataset with Claim-Premise Relations from Maria Poiaganova

In [ ]:
# Retrieved from https://github.com/mpoiaganova/political-argument-mining
utest

,Sentence,Label,Components
0,"Once again, since the last briefing to the Cou...",Claim,{'the situation regarding Ukraine has seriousl...
3,"Following close to two weeks of relative calm,...",Premise,{'Following close to two weeks of relative cal...
4,The individuals involved called for secession ...,Premise,{'The individuals involved called for secessio...
5,"In Donetsk, some of these individuals declared...",Premise,"{""In Donetsk, some of these individuals declar..."
6,It has now been reported that there have been ...,Premise,{'It has now been reported that there have bee...
...,...,...,...
4746,We welcome Italy's decision to designate dialo...,Claim,"{""We welcome Italy's decision to designate dia..."
4747,China supports practical and effective coopera...,Claim,{'China supports practical and effective coope...
4748,We welcome all the positive efforts being made...,Claim,{'We welcome all the positive efforts being ma...
4749,We hope that all the parties concerned will wo...,Claim,{'We hope that all the parties concerned will ...


In [ ]:
# Creating data loader
unsc_m_loader = DataLoader(utest_set_set, batch_size=16)

In [ ]:
# Formating
utest_set_set.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
# Getting predictions
unsc_m_predictions = predict(model, unsc_m_loader)

# Converting predictions to flat list
unsc_m_predictions = [item for sublist in unsc_m_predictions for item in sublist]

# Adding prections to test set Dataframe
utest_df['Predicted_Label'] = unsc_m_predictions

In [ ]:
utest_df

,Speech,Label,Components,Component,Predicted_Label
0,"Once again, since the last briefing to the Cou...",Claim,{'the situation regarding Ukraine has seriousl...,0,1
3,"Following close to two weeks of relative calm,...",Premise,{'Following close to two weeks of relative cal...,1,1
4,The individuals involved called for secession ...,Premise,{'The individuals involved called for secessio...,1,1
5,"In Donetsk, some of these individuals declared...",Premise,"{""In Donetsk, some of these individuals declar...",1,1
6,It has now been reported that there have been ...,Premise,{'It has now been reported that there have bee...,1,1
...,...,...,...,...,...
4746,We welcome Italy's decision to designate dialo...,Claim,"{""We welcome Italy's decision to designate dia...",0,0
4747,China supports practical and effective coopera...,Claim,{'China supports practical and effective coope...,0,0
4748,We welcome all the positive efforts being made...,Claim,{'We welcome all the positive efforts being ma...,0,0
4749,We hope that all the parties concerned will wo...,Claim,{'We hope that all the parties concerned will ...,0,0


In [ ]:
# Getting true labels and predicted labels form test dataframe
y_true_u_m = utest_df['Component']
y_pred_u_m = utest_df['Predicted_Label']

In [ ]:
# Calculating metrics
accuracy_u_m = accuracy_score(y_true_u_m, y_pred_u_m)
precision_u_m = precision_score(y_true_u_m, y_pred_u_m, pos_label=1)  # assuming 'Premise'=1 is the positive class
recall_u_m = recall_score(y_true_u_m, y_pred_u_m, pos_label=1)
f1_u_m = f1_score(y_true_u_m, y_pred_u_m, pos_label=1)
conf_matrix_u_m = confusion_matrix(y_true_u_m, y_pred_u_m)

In [ ]:
# Printing the metrics
print("Accuracy:", accuracy_u_m)
print("Precision:", precision_u_m)
print("Recall:", recall_u_m)
print("F1 Score:", f1_u_m)
print("Confusion Matrix:\n", conf_matrix_u_m)
print("\nClassification Report:\n", classification_report(y_true_u_m, y_pred_u_m))

Accuracy: 0.7325642370214998
Precision: 0.7155742633794347
Recall: 0.6850892343120323
F1 Score: 0.7
Confusion Matrix:
 [[1604  473]
 [ 547 1190]]

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.77      0.76      2077
           1       0.72      0.69      0.70      1737

    accuracy                           0.73      3814
   macro avg       0.73      0.73      0.73      3814
weighted avg       0.73      0.73      0.73      3814



# Testing on UNSCUkrArg dataset

In [ ]:
# Loading annotated dataset
data_unsc = pd.read_excel("/content/Arg. Schemes UNSC Corpus.xlsx")
data_unsc

,fileid,filename,speaker,speech_number,sentence,sentence token,sentence character offset begin,sentence character offset end,annotated argument,label,labels,argument ID,argumentation scheme,relation,argument target,argument character offset begin,argument character offset end,related sentences
0,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,Let me thank the Nigerian presidency for conve...,"['Let', 'me', 'thank', 'the', 'Nigerian', 'pre...",0,155,[],no_component,[],[],[],[],[],[],[],[]
1,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,I might refer to some of the comments by our c...,"['I', 'might', 'refer', 'to', 'some', 'of', 't...",156,239,[],no_component,[],[],[],[],[],[],[],[]
2,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,I would also like to thank Under Secretary-Gen...,"['I', 'would', 'also', 'like', 'to', 'thank', ...",240,324,[],no_component,[],[],[],[],[],[],[],[]
3,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,"As so many of us have said here, the internati...","['As', 'so', 'many', 'of', 'us', 'have', 'said...",326,507,[],no_component,[],[],[],[],[],[],[],[]
4,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,"But, as we know today, the Geneva statement is...","['But', ',', 'as', 'we', 'know', 'today', ',',...",508,908,['the Geneva statement is just another documen...,claim,"['claim', 'premise']","[2, 1]","['no scheme', 'evidence']","[[], [(2, 'evidence', 'Russia')]]","['no target', 'Russia']","[531, 615]","[614, 908]",['the Geneva statement is just another documen...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,12,We hope that the relevant parties will remain ...,"['We', 'hope', 'that', 'the', 'relevant', 'par...",168,292,[],no_component,[],[],[],[],[],[],[],[]
1492,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,12,Settling the question of Ukraine involves the ...,"['Settling', 'the', 'question', 'of', 'Ukraine...",294,378,['Settling the question of Ukraine involves th...,premise,['premise'],[2],['positive_value'],"[[(1, 'positive_value', 'Other')]]",['Other'],[294],[378],['It should be considered in a balanced manner.']
1493,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,12,It should be considered in a balanced manner.,"['It', 'should', 'be', 'considered', 'in', 'a'...",379,424,['It should be considered in a balanced manner.'],claim,['claim'],[1],['no scheme'],[[]],['no target'],[379],[424],[]
1494,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,12,Political and diplomatic channels should conti...,"['Political', 'and', 'diplomatic', 'channels',...",425,579,['Political and diplomatic channels should con...,claim,"['claim', 'premise']","[3, 4]","['no scheme', 'positive_consequence']","[[], [(3, 'positive_consequence', 'Other'), (5...","['no target', 'Other, Other']","[425, 489]","[488, 579]",['Political and diplomatic channels should con...


In [ ]:
# Mapping label values to numeric values
label_mapping = {"premise": 1, "claim": 0}

data_unsc_filtered['label'] = data_unsc_filtered['label'].map(label_mapping)
data_unsc_filtered

<ipython-input-81-639a0348e4d3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_unsc_filtered['label'] = data_unsc_filtered['label'].map(label_mapping)


,fileid,filename,speaker,speech_number,sentence,sentence token,sentence character offset begin,sentence character offset end,annotated argument,label,labels,argument ID,argumentation scheme,relation,argument target,argument character offset begin,argument character offset end,related sentences
4,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,"But, as we know today, the Geneva statement is...","['But', ',', 'as', 'we', 'know', 'today', ',',...",508,908,['the Geneva statement is just another documen...,0,"['claim', 'premise']","[2, 1]","['no scheme', 'evidence']","[[], [(2, 'evidence', 'Russia')]]","['no target', 'Russia']","[531, 615]","[614, 908]",['the Geneva statement is just another documen...
5,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,"Despite growing provocations, the Ukrainian au...","['Despite', 'growing', 'provocations', ',', 't...",910,1041,"['Despite growing provocations, the Ukrainian ...",0,['claim'],[10],['no scheme'],[[]],['no target'],[910],[1041],[]
6,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,Barricades in Maidan Square are being dismantled.,"['Barricades', 'in', 'Maidan', 'Square', 'are'...",1042,1091,['Barricades in Maidan Square are being disman...,1,['premise'],[3],['evidence'],"[[(10, 'evidence', 'Ukraine')]]",['Ukraine'],[1042],[1091],"['Despite growing provocations, the Ukrainian ..."
7,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,I recall my French colleague referring a numbe...,"['I', 'recall', 'my', 'French', 'colleague', '...",1092,1176,['I recall my French colleague referring a num...,1,['premise'],[4],['evidence'],"[[(10, 'evidence', 'Ukraine')]]",['Ukraine'],[1092],[1176],"['Despite growing provocations, the Ukrainian ..."
8,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,16,Users of those will see that there is a signif...,"['Users', 'of', 'those', 'will', 'see', 'that'...",1177,1293,['Users of those will see that there is a sign...,1,['premise'],[5],['evidence'],"[[(10, 'evidence', 'Ukraine')]]",['Ukraine'],[1177],[1293],"['Despite growing provocations, the Ukrainian ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488,UNSC_2014_SPV.7154_spch021,UNSC_2014_SPV.7154_spch021_sentsplit_Russian_F...,Federation,21,That needs to be avoided.,"['That', 'needs', 'to', 'be', 'avoided']",7800,7825,['That needs to be avoided.'],0,['claim'],[37],['no scheme'],[[]],['no target'],[7800],[7825],[]
1492,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,12,Settling the question of Ukraine involves the ...,"['Settling', 'the', 'question', 'of', 'Ukraine...",294,378,['Settling the question of Ukraine involves th...,1,['premise'],[2],['positive_value'],"[[(1, 'positive_value', 'Other')]]",['Other'],[294],[378],['It should be considered in a balanced manner.']
1493,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,12,It should be considered in a balanced manner.,"['It', 'should', 'be', 'considered', 'in', 'a'...",379,424,['It should be considered in a balanced manner.'],0,['claim'],[1],['no scheme'],[[]],['no target'],[379],[424],[]
1494,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,12,Political and diplomatic channels should conti...,"['Political', 'and', 'diplomatic', 'channels',...",425,579,['Political and diplomatic channels should con...,0,"['claim', 'premise']","[3, 4]","['no scheme', 'positive_consequence']","[[], [(3, 'positive_consequence', 'Other'), (5...","['no target', 'Other, Other']","[425, 489]","[488, 579]",['Political and diplomatic channels should con...


In [ ]:
# Renaming column
data_unsc_filtered = data_unsc_filtered.rename(columns={"sentence": "Speech"})

NameError: name 'data_unsc_filtered' is not defined

In [ ]:
# Tokenizing function
def tokenize_function(examples):
    return tokenizer(examples['Sentence'], padding="max_length", truncation=True)

In [ ]:
# Converting to readeble dataset for model
unsc_test = Dataset.from_pandas(data_unsc_filtered)

In [ ]:
unsc_test

Dataset({
    features: ['fileid', 'filename', 'speaker', 'speech_number', 'Speech', 'sentence token', 'sentence character offset begin', 'sentence character offset end', 'annotated argument', 'label', 'labels', 'argument ID', 'argumentation scheme', 'relation', 'argument target', 'argument character offset begin', 'argument character offset end', 'related sentences', '__index_level_0__'],
    num_rows: 946
})

In [ ]:
# Tokenizing test set
unsc_tokenized_test = unsc_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/946 [00:00<?, ? examples/s]

In [ ]:
# Formating tokenized set
unsc_tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
# Creating data loader
arg_unsc_loader = DataLoader(unsc_tokenized_test, batch_size=16)

In [ ]:
# Getting predictions
unsc_predictions = predict(model, arg_unsc_loader)

# COnverting predictions to flat list
unsc_predictions = [item for sublist in unsc_predictions for item in sublist]

# Adding prections to test set Dataframe
data_unsc_filtered['Predicted_Label'] = unsc_predictions

In [ ]:
# Getting true labels and predicted labels form test dataframe
y_true_unsc = data_unsc_filtered['label']
y_pred_unsc = data_unsc_filtered['Predicted_Label']

# Calculating metrics
accuracy_unsc = accuracy_score(y_true_unsc, y_pred_unsc)
precision_unsc = precision_score(y_true_unsc, y_pred_unsc, pos_label=1)  # assuming 'Arg'=1 is the positive class
recall_unsc = recall_score(y_true_unsc, y_pred_unsc, pos_label=1)
f1_unsc = f1_score(y_true_unsc, y_pred_unsc, pos_label=1)
conf_matrix_unsc = confusion_matrix(y_true_unsc, y_pred_unsc)


In [ ]:
# Printing the metrics
print("Accuracy:", accuracy_unsc)
print("Precision:", precision_unsc)
print("Recall:", recall_unsc)
print("F1 Score:", f1_unsc)
print("Confusion Matrix:\n", conf_matrix_unsc)
print("\nClassification Report:\n", classification_report(y_true_unsc, y_pred_unsc))

Accuracy: 0.7304439746300211
Precision: 0.8141414141414142
Recall: 0.7120141342756183
F1 Score: 0.7596606974552309
Confusion Matrix:
 [[288  92]
 [163 403]]

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.76      0.69       380
           1       0.81      0.71      0.76       566

    accuracy                           0.73       946
   macro avg       0.73      0.73      0.73       946
weighted avg       0.74      0.73      0.73       946



In [ ]:
# Saving results
data_unsc_filtered.to_excel("Task2_results_UNSCUkrArg_Dataset.xlsx")

# Classifying Debate 7138

In [ ]:
# Loading data sample of Debate 7138
unsc_arg_app = pd.read_excel("/content/7138_speches_with_Task1_predictions.xlsx")
unsc_arg_app

,File_name,Speech,label,Attitude Fragment,Predicted Argument or Not
0,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Some colleagues today have achieved high level...,none,NaN,1
1,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I must mention that the Ukrainian colleague ne...,judgement_positive,I must mention,1
2,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"Ifone is to speak ofblood, indeed the blood is...",judgement_negative,henchmen,1
3,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"With regard to other statements, I must note t...",none,NaN,0
4,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Colleagues that have taken the floor discussed...,judgement_negative,distorted,1
...,...,...,...,...,...
347,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Our willing submission to the rule of the Inte...,none,NaN,1
348,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Nigeria is vehemently opposed to unilateral se...,affect_negative,is vehemently opposed to,1
349,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,"Standing on that fundamental principle, Nigeri...",affect_negative,is opposed to,1
350,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,It is therefore needless to restate that the i...,judgement_negative,It is therefore needless to restate,1


In [ ]:
unsc_arg_app.columns

Index(['File_name', 'Speech', 'label', 'Attitude Fragment',
       'Predicted Argument or Not'],
      dtype='object')

In [ ]:
# Mapping labels to numeric values
label_mapping = {1: "Arg", 0: "Non_Arg"}
unsc_arg_app['Predicted Argument'] = unsc_arg_app['Predicted Argument or Not'].map(label_mapping)
unsc_arg_app

,File_name,Speech,label,Attitude Fragment,Predicted Argument or Not,Predicted Argument
0,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Some colleagues today have achieved high level...,none,NaN,1,Arg
1,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I must mention that the Ukrainian colleague ne...,judgement_positive,I must mention,1,Arg
2,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"Ifone is to speak ofblood, indeed the blood is...",judgement_negative,henchmen,1,Arg
3,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"With regard to other statements, I must note t...",none,NaN,0,O
4,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Colleagues that have taken the floor discussed...,judgement_negative,distorted,1,Arg
...,...,...,...,...,...,...
347,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Our willing submission to the rule of the Inte...,none,NaN,1,Arg
348,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Nigeria is vehemently opposed to unilateral se...,affect_negative,is vehemently opposed to,1,Arg
349,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,"Standing on that fundamental principle, Nigeri...",affect_negative,is opposed to,1,Arg
350,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,It is therefore needless to restate that the i...,judgement_negative,It is therefore needless to restate,1,Arg


In [ ]:
unsc_app_args_filtered = unsc_arg_app[unsc_arg_app['Predicted Argument'] == 'Arg']

In [ ]:
unsc_app_args_filtered['Predicted Argument'].value_counts()

,count
Predicted Argument,
Arg,315


In [ ]:
unsc_app_args_filtered

,File_name,Speech,label,Attitude Fragment,Predicted Argument or Not,Predicted Argument
0,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Some colleagues today have achieved high level...,none,NaN,1,Arg
1,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I must mention that the Ukrainian colleague ne...,judgement_positive,I must mention,1,Arg
2,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"Ifone is to speak ofblood, indeed the blood is...",judgement_negative,henchmen,1,Arg
4,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Colleagues that have taken the floor discussed...,judgement_negative,distorted,1,Arg
5,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I will not comment on everything heard today -...,judgement_negative,unacceptable,1,Arg
...,...,...,...,...,...,...
347,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Our willing submission to the rule of the Inte...,none,NaN,1,Arg
348,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Nigeria is vehemently opposed to unilateral se...,affect_negative,is vehemently opposed to,1,Arg
349,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,"Standing on that fundamental principle, Nigeri...",affect_negative,is opposed to,1,Arg
350,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,It is therefore needless to restate that the i...,judgement_negative,It is therefore needless to restate,1,Arg


In [ ]:
df_unsc7138 = unsc_app_args_filtered[['File_name', 'Speech']]
df_unsc7138

,File_name,Speech
0,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Some colleagues today have achieved high level...
1,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I must mention that the Ukrainian colleague ne...
2,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"Ifone is to speak ofblood, indeed the blood is..."
4,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Colleagues that have taken the floor discussed...
5,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I will not comment on everything heard today -...
...,...,...
347,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Our willing submission to the rule of the Inte...
348,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Nigeria is vehemently opposed to unilateral se...
349,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,"Standing on that fundamental principle, Nigeri..."
350,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,It is therefore needless to restate that the i...


In [ ]:
test_unsc7138 = Dataset.from_pandas(df_unsc7138)
test_unsc7138

Dataset({
    features: ['File_name', 'Speech', '__index_level_0__'],
    num_rows: 315
})

In [ ]:
tokenized_unsc7138_set = test_unsc7138.map(tokenize_function, batched=True)

Map:   0%|          | 0/315 [00:00<?, ? examples/s]

In [ ]:
tokenized_unsc7138_set

Dataset({
    features: ['File_name', 'Speech', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 315
})

In [ ]:
tokenized_unsc7138_set.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
tokenized_unsc7138_set

Dataset({
    features: ['File_name', 'Speech', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 315
})

In [ ]:
# Creating data loader
test_loader_unsc7138 = DataLoader(tokenized_unsc7138_set, batch_size=16)

# Getting predictions
predictions_unsc7138 = predict(model, test_loader_unsc7138)

# Converting predictions to flat list
unsc7138_predictions = [item for sublist in predictions_unsc7138 for item in sublist]

# Adding prections to test set Dataframe
unsc_app_args_filtered['Predicted_Labels'] = unsc7138_predictions

<ipython-input-80-849b5d97042b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsc_app_args_filtered['Predicted_Labels'] = unsc7138_predictions


In [ ]:
unsc_app_args_filtered

,File_name,Speech,label,Attitude Fragment,Predicted Argument or Not,Predicted Argument,Predicted_Labels
0,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Some colleagues today have achieved high level...,none,NaN,1,Arg,1
1,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I must mention that the Ukrainian colleague ne...,judgement_positive,I must mention,1,Arg,0
2,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"Ifone is to speak ofblood, indeed the blood is...",judgement_negative,henchmen,1,Arg,1
4,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Colleagues that have taken the floor discussed...,judgement_negative,distorted,1,Arg,1
5,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I will not comment on everything heard today -...,judgement_negative,unacceptable,1,Arg,0
...,...,...,...,...,...,...,...
347,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Our willing submission to the rule of the Inte...,none,NaN,1,Arg,0
348,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Nigeria is vehemently opposed to unilateral se...,affect_negative,is vehemently opposed to,1,Arg,0
349,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,"Standing on that fundamental principle, Nigeri...",affect_negative,is opposed to,1,Arg,0
350,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,It is therefore needless to restate that the i...,judgement_negative,It is therefore needless to restate,1,Arg,0


In [ ]:
# Mapping labels to numeric values
label_mapping_prem_cla = {1: "Premise", 0: "Claim"}
unsc_app_args_filtered["claim_premise_predictions"] = unsc_app_args_filtered["Predicted_Labels"].map(label_mapping_prem_cla)
unsc_app_args_filtered

<ipython-input-84-4568c5464233>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsc_app_args_filtered["claim_premise_predictions"] = unsc_app_args_filtered["Predicted_Labels"].map(label_mapping_prem_cla)


,File_name,Speech,label,Attitude Fragment,Predicted Argument or Not,Predicted Argument,Predicted_Labels,claim_premise_predictions
0,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Some colleagues today have achieved high level...,none,NaN,1,Arg,1,Premise
1,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I must mention that the Ukrainian colleague ne...,judgement_positive,I must mention,1,Arg,0,Claim
2,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,"Ifone is to speak ofblood, indeed the blood is...",judgement_negative,henchmen,1,Arg,1,Premise
4,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,Colleagues that have taken the floor discussed...,judgement_negative,distorted,1,Arg,1,Premise
5,UNSC_2014_SPV.7138_spch020_sentsplit_Russian_F...,I will not comment on everything heard today -...,judgement_negative,unacceptable,1,Arg,0,Claim
...,...,...,...,...,...,...,...,...
347,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Our willing submission to the rule of the Inte...,none,NaN,1,Arg,0,Claim
348,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,Nigeria is vehemently opposed to unilateral se...,affect_negative,is vehemently opposed to,1,Arg,0,Claim
349,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,"Standing on that fundamental principle, Nigeri...",affect_negative,is opposed to,1,Arg,0,Claim
350,UNSC_2014_SPV.7138_spch014_sentsplit_Nigeria,It is therefore needless to restate that the i...,judgement_negative,It is therefore needless to restate,1,Arg,0,Claim


In [ ]:
unsc_app_args_filtered['claim_premise_predictions'].value_counts()

,count
claim_premise_predictions,
Claim,205
Premise,110


In [ ]:
# Saving results
unsc_app_args_filtered.to_excel("Task2_results_Debate_7138.xlsx")